In [1]:
# !pip install pyvis==0.3.1
# !pip install pyaml

In [2]:
from typing import List, Dict, Any
from itertools import count
from dataclasses import dataclass, field
from pyvis.network import Network as PyvisNetwork
import random
import os
import shutil
import yaml
from itertools import product
import numpy as np

class Default:
    def print(self):
        fields = [(attribute, value) for attribute, value in self.__dict__.items()]
        print('-'*30, f'{self.__class__.__name__}', '-'*30)
        for field in self.__dict__.items():
            test = field[0]
            print(f"{field[0]}: {field[1]}")

@dataclass()
class Node(Default):
    id: int = field(default_factory=lambda count_=count(): next(count_), init=False)
    title: str = field(default_factory=str)
    description: str = field(default_factory=str)
    relations: List[int] = field(default_factory=list[int])
    # relations: List['Node'] = field(default_factory=list)
    metadata: Dict[str, Any] = field(default_factory=dict)

@dataclass()
class Edge(Default):
    id: int = field(default_factory=lambda count_=count(): next(count_), init=False)
    title: str = field(default_factory=str)
    description: str = field(default_factory=str)
    tail: 'Node' = field(default_factory='Node')
    head: 'Node' = field(default_factory='Node')
    # relations: List['Edge'] = field(default_factory=list)
    metadata: Dict[str, Any] = field(default_factory=dict)
    
    def __post_init__(self):
        self.tail.relations.append(self.head.id)
        self.head.relations.append(self.tail.id)

@dataclass()
class Graph(Default):
    nodes: List['Node'] = field(default_factory=list['Node'])
    edges: List['Edge'] = field(default_factory=list['Edge'])

    def from_yaml(self, file_path: str = './data/graphs/test_1.yaml'):
        with open(file_path) as file:
            loaded = yaml.load(file, Loader=yaml.FullLoader)
            for node in loaded['NODES']:
                self.nodes.append(Node(title=node))
            for edge in loaded['EDGES']:
                self.edges.append(Edge(head=self.get_node(edge.get('head')), tail=self.get_node(edge.get('tail')), title=edge.get('title')))

    def get_node(self, title: str):
        for node in self.nodes:
            if node.title == title:
                return node
        return None

    def from_path(self, path:str):
        path = path.split(',')
        self.nodes = [Node(title=name) for name in path]
        self.edges = []
        print(path)
        for i in range(0, len(path) - 1):
            self.edges.append(Edge(head=self.get_node(path[i]), tail=self.get_node(path[i+1])))

    def from_adjacency_matrix(self, adj_matrix):
        self.nodes = [Node(title=i) for i in range(len(adj_matrix))]
        self.edges = []

        for i in range(len(adj_matrix)):
            for j in range(i+1, len(adj_matrix)):
                if adj_matrix[i][j] == 1:
                    self.edges.append(Edge(head=self.nodes[i], tail=self.nodes[j]))


    def adjacency_matrix_to_yaml(self, adj_matrix, file_path):
        nodes = [i for i in range(len(adj_matrix))]
        edges = []

        for i in range(len(adj_matrix)):
            for j in range(i, len(adj_matrix)):
                if adj_matrix[i][j] == 1:
                    edges.append({'head': nodes[i], 'tail': nodes[j]})

        graph_yaml = {'NODES': nodes, 'EDGES': edges}

        with open(file_path, 'w') as yaml_file:
            yaml.dump(graph_yaml, yaml_file, default_flow_style=False, sort_keys=False)


    def save_to_html(self, filename="data/network.html", proxied=False, remove_lib=False, directed=False):
        if filename.startswith('./'):
            filename = filename[2:]
        if filename.startswith('/'):
            filename = filename[1:]
        # net = PyvisNetwork(directed=directed, width="1920px", height="1080px", bgcolor="#222222")
        net = PyvisNetwork(directed=directed, width="1920px", height="1080px", bgcolor="#eeeeee")
        proxy_address = "https://vscode.kripso-world.com/proxy/5501"
        proxy_address = "http://127.0.0.1:5500"

        for node in self.nodes:
            hexadecimal = ["#"+''.join([random.choice('ABCDEF0123456789') for i in range(6)])][0]
            net.add_node(node.id, label=str(node.title), shape="circle", color=hexadecimal)

        for edge in self.edges:
            net.add_edge(edge.head.id, edge.tail.id, title=edge.title, label=edge.title)
        net.force_atlas_2based()
        net.repulsion(
            node_distance=200,
            central_gravity=0.2,
            spring_length=100,
            spring_strength=0.05,
            damping=0.09
        )

        os.makedirs('./data', exist_ok=True)

        net.set_edge_smooth('dynamic')
        net.show(f"./{filename}")
        
        if remove_lib and os.path.isdir('./lib'):
            shutil.rmtree('./lib')
        
        if proxied:
            print(f"{proxy_address}/{filename}")
        else:
            print(f"./{filename}")

# calculate number of hamiltonial paths in prism graphs

In [3]:
import math

for n in range(3,25,1):
    print(4*n*(math.floor((1/2) * (n ** 2)) + 1))

60
144
260
456
700
1056
1476
2040
2684
3504
4420
5544
6780
8256
9860
11736
13756
16080
18564
21384
24380
27744


# visualisation

In [4]:
import os

file_paths = os.listdir('../data/graphs/')

def get_paths(n, k):
    calculations = []
    for path in file_paths:
        with open(f"../data/graphs/{path}") as file:
            loaded = yaml.load(file, Loader=yaml.FullLoader)
            n_ = None
            k_ = None
            start_position = ''
            for kwarg_ in path.split(';'):
                kwarg_ = kwarg_.replace('.yaml', '')
                key, value = kwarg_.split('=')
                if key == 'n':
                    n_ = value
                if key == 'k':
                    k_ = value
                if key == 'start_position':
                    start_position = value
            if int(n_) == n and int(k_) == k:
                calculations.append({'start_position': start_position,'paths': loaded['paths'], 'combinations_raw': len(loaded['paths']), 'combinations_multiplied': loaded['count']})
    return calculations


In [6]:
paths = get_paths(15,5)

start_position = random.randint(0,4)
random_path = paths[start_position]['paths'][random.randint(1, len(paths[start_position]['paths']))]

In [7]:
posible_hamiltinial_paths = 0
for path in paths:
    posible_hamiltinial_paths += path['combinations_multiplied']
print('path lenght:', len(random_path.split(',')))
print('hamiltonial paths:', posible_hamiltinial_paths)

path lenght: 30
hamiltonial paths: 11700


In [9]:
new_graph = Graph()
new_graph.from_path(random_path)
new_graph.save_to_html(filename='./data/network.html', directed=False, remove_lib=True, proxied=True)

['0', '15', '20', '25', '10', '9', '24', '19', '29', '14', '13', '28', '18', '23', '8', '7', '6', '5', '4', '3', '2', '1', '16', '21', '26', '11', '12', '27', '22', '17']
http://127.0.0.1:5500/data/network.html
